<a href="https://colab.research.google.com/github/xivh/stylegan2-colab-notebooks/blob/master/Convert_images_to_shards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

# Get the converter

In [ ]:
!git clone https://github.com/shawwn/danbooru-tools

# Get your images



For example, get your zipped, processed images from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Change the first argument to the path of your zip file - you can find it in the side browser and right click to copy the path

In [ ]:
!cp "zip_file_path" /content/

If your file was zipped, you need to unzip it

In [ ]:
!mkdir images/
!unzip your_zip_file.zip -d images/

You want your images to be in images/\*.jpg, not images/another_folder/\*.jpg

# Convert your images to shards

Shards are used to shuffle the images. The minimum is 8 (8 tpu cores), the default is 2048. I have set it to 128 below, but you can change it if you want. You may get fewer shards than you set because each shard must have the same number of images.

In [ ]:
!find images/ -type f -name '*.jpg' > images.txt
!python danbooru-tools/danbooru_to_tfrecord.py --name images --out 'your_output_folder' --files images.txt --shards 128

Upload to Google bucket


1.   Authenticate yourself. This one might not work... 
2.   Make the folder if it doesn't exist (you could do this through the GUI)
3.   Upload your files



In [ ]:
!gcloud auth login
!gcloud config set project your_project_id # change to your project

In [ ]:
# required if dataset folder does not exist
!touch "$your_output_folder"/GSUTIL-IS-NOT-A-FILESYSTEM.txt
!gsutil cp "$your_output_folder"/GSUTIL-IS-NOT-A-FILESYSTEM.txt "$your_google_bucket_path"/GSUTIL-IS-NOT-A-FILESYSTEM.txt
!rm "$your_output_folder"/GSUTIL-IS-NOT-A-FILESYSTEM.txt

In [ ]:
!gsutil -m cp your_output_folder/* gs://your_google_bucket_path

# Extra: make the necessary tfrecords

Currently, you need to make a dummy folder with empty tfrecords

In [ ]:
dataset = "your_dataset_name" # set this to your dataset name

from pathlib import Path

Path("dummy_data", dataset).mkdir(parents=True, exist_ok=True)

for i in range(2, 11):
  Path("dummy_data", dataset, dataset + "-r%02d.tfrecords" % i).touch()

Assuming you are already authenticated (see previous sections), upload

In [ ]:
!gsutil -m cp -r dummy_data/ gs://your_google_bucket/dummy_data

# Extra: Quickly process your images

Only do this for a quick check - you should have already processed your images before using this notebook

Your original images will not be preserved, so if you mess up, you'll have to download them again (or you could make a copy of the folder)

In [ ]:
!pip install Augmentor
!pip install fastai

Check the number of images

In [ ]:
!ls images | wc -l

Resize all your images to the right dimensions



In [ ]:
size = 512 # set this - I am making squares

import Augmentor
p = Augmentor.Pipeline("images")
p.resize(probability=1, height=size, width=size, resample_filter="ANTIALIAS")
p.process()

Move the output images back up to the images folder and remove the original images

In [ ]:
!rm images/*.jpg
!mv images/output/* images/
!rm images/output -d

Check the images to make sure they all open

In [ ]:
from fastai.vision import *
verify_images("images", delete=True)

Check how many images you have left - hopefully about the same, definitely not more

In [ ]:
!ls images | wc -l